### Get data from Steam Community Market API and save to csv

In [35]:
# Obtaining the data + inputting items seeking for

import re
import requests
import pandas as pd

dailyCookie = "76561199704981720%7C%7CeyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MEVGQV8yNDlCQUM3RV9GOUNBOSIsICJzdWIiOiAiNzY1NjExOTk3MDQ5ODE3MjAiLCAiYXVkIjogWyAid2ViOmNvbW11bml0eSIgXSwgImV4cCI6IDE3MTkyNDU3OTcsICJuYmYiOiAxNzEwNTE3ODM5LCAiaWF0IjogMTcxOTE1NzgzOSwgImp0aSI6ICIwRUY1XzI0OUJBQzhGXzZCODk5IiwgIm9hdCI6IDE3MTg5OTc2MjQsICJydF9leHAiOiAxNzM3NTA1ODQ4LCAicGVyIjogMCwgImlwX3N1YmplY3QiOiAiODQuNjQuMTAxLjIyNiIsICJpcF9jb25maXJtZXIiOiAiNzguODYuMzIuMTc0IiB9.sELZ0bQkVyW4gVpI0Ps41957Ce4s51Z_aSEa-vdk2y2--_MylLypcxcuVT0YotiFxStllmvhyyoxp4cN7MOyCQ"
items = ["Glove Case Key", "Officer Jacques Beltram | Gendarmerie Nationale", "Kilowatt Case", "AK-47 | Blue Laminate (Factory New)", "Glove Case"]

def fetch_item_to_df(item):
    # get historical price data of item from API
    url = "https://steamcommunity.com/market/pricehistory/"
    params = {
        'country': 'US',
        'currency': '1',
        'appid': '730',
        'market_hash_name': item
    }
    cookies = {'steamLoginSecure': dailyCookie}

    response = requests.get(url, params=params, cookies=cookies)
    jsonData = response.json()

    # print error message if request failed
    if response.status_code != 200:
        print(f"Failed to fetch data for {item}. Status code: {response.status_code}")
        return None
    
    # convert and clean data to dataframe object
    price_history = jsonData['prices']
    price_history_df = pd.DataFrame(price_history, columns=['date', 'price_usd', 'volume'])
    price_history_df['date'] = pd.to_datetime(price_history_df['date'].str[0:-4], format='%b %d %Y %H')
    price_history_df['volume'] = pd.to_numeric(price_history_df['volume'])
    price_history_df.set_index('date', inplace=True)
    return price_history_df


def sanitize_filename(filename):
    """Sanitizes the filename to ensure it is valid for most operating systems."""
    filename = re.sub(r'[\\/*?:"<>|]', '_', filename)  # Replace disallowed characters with underscore
    filename = re.sub(r'\s+', '_', filename)  # Replace spaces with underscores
    return filename

def save_item_to_csv(item):
    csvData = './data/'+ sanitize_filename(item) +'.csv'
    fetch_item_to_df(item).to_csv(csvData, index=True)

#0     Nov 29 2016 01: +0      2.017   5261 - original format
## fetch and save data in items to csv - Uncomment this when checking new item or updating previous datasets

# for index, item in enumerate(items):
#     save_item_to_csv(item)






                     price_usd  volume
date                                  
2016-11-29 01:00:00      2.017    5261
2016-11-30 01:00:00      2.186    3776
2016-12-01 01:00:00      2.192    3207
2016-12-02 01:00:00      2.181    4250
2016-12-03 01:00:00      2.170    5381
...                        ...     ...
2024-06-22 06:00:00     14.591       1
2024-06-22 07:00:00     14.417       7
2024-06-22 16:00:00     14.575       1
2024-06-22 23:00:00     14.591       4
2024-06-23 00:00:00     14.593       1

[2808 rows x 2 columns]


,price_usd,volume
date,,
2016-11-29 01:00:00,2.017,5261
2016-11-30 01:00:00,2.186,3776
2016-12-01 01:00:00,2.192,3207
2016-12-02 01:00:00,2.181,4250
2016-12-03 01:00:00,2.170,5381
...,...,...
2024-06-22 06:00:00,14.591,1
2024-06-22 07:00:00,14.417,7
2024-06-22 16:00:00,14.575,1


### Item Verification

In [20]:
# Call the method (From Get Data notebook) to get current_item dataframe

from matplotlib import pyplot as plt

current_item = fetch_item_to_df(items[4])

# shows what current item is, note: Array item stats with 0.
print(items[4])
print(current_item.head())



Glove Case
            price_usd  volume
date                         
2016-11-29      6.457   53686
2016-11-30      3.576   49360
2016-12-01      2.717   48668
2016-12-02      2.083   54474
2016-12-03      1.616   64918


### Historic Price (0)  

In [21]:
# > Historic Price (Basic)


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['price_usd'], mode='lines+markers', name='Price USD'))

    # Formatting
    fig.update_layout(
        title='Median Price over time (Historic Chart)',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True

        )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=current_item.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=current_item.index[-1], disabled=False)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)





interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_graph(start_date, end_date)>

### Volume Indicatior (1)

In [22]:
# > Volume Indicatior (1)

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Bar(x=filtered_data.index, y=filtered_data['volume'], name='Price USD'))

    # Formatting
    fig.update_layout(
        title='Volume Chart',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True
    )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=current_item.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=current_item.index[-1], disabled=False)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)



interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_graph(start_date, end_date)>

### Moving Averages (2)

In [23]:
# # SIMPLE Moving Average (SMA)                            
#  | Part of Moving Averages (2) block


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# calculates the 7-day moving average. 
# change window = to change period for average.
df['smoothened_price'] = df['price_usd'].rolling(window=2).mean()

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['smoothened_price'], mode='lines', name='Smoothened Price'))

    # Formatting
    fig.update_layout(
        title='Simple Moving Average Chart',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True

        )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=current_item.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=current_item.index[-1], disabled=False)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)


interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_graph(start_date, end_date)>

In [24]:
 # SIMPLE Moving Average (SMA) -  CANDLESTICK  
 # | Part of Moving Averages (2) block

# NOTE: this produces candlesticks based on (specified time frame e.g. Weekly) data


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# Resample the data to weekly timeframe
# NOTE: Range can be altered: 'W' (week), 'M' (Month), '2D' (2 Days)
new_data = df.resample('5M').agg({
    'price_usd': ['max', 'min', 'mean'],
    'volume': 'sum'
})

# Flatten the multi-level columns and rename
new_data.columns = ['high', 'low', 'mean', 'volume']
# print(new_data.head(10))

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):


    mask = (new_data.index >= pd.to_datetime(start_date)) & (new_data.index <= pd.to_datetime(end_date))
    filtered_data = new_data.loc[mask]

    
    # Creating the candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['mean'],
                                         high=filtered_data['high'],
                                         low=filtered_data['low'],
                                         close=filtered_data['mean'],
                                         name='Candlestick')])
    


    # Formatting
    fig.update_layout(
        title='SMA - Candlestick',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        
        showlegend=True
    )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=current_item.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=current_item.index[-1], disabled=False)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)


interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_graph(start_date, end_date)>

In [25]:
#  EXPONENTIAL Moving Average (EMA)
# | Part of Moving Averages (2) block


# !!!!!!!!!!!!!!!!!! NOTE: NOT IMPLMENTED !!!!!!!!!!!!!!!!!!!!!!!!



